In [1]:
%load_ext autoreload

In [2]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction


from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Sporadically need to run `pkill -f meshcat`

from symbolic_parsing_helpers import generate_rationalize_trig_expr_rules
from symbolic_parsing_helpers import xreplace
from symbolic_parsing_helpers import NotRationalFunctionException
# from symbolic_parsing_helpers import rationalize_trig_expr
from pydrake.all import RationalFunction

# Build plant and simulation

In [3]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]


In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()


diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)


def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)


def eval_cons_rational(t0, t1, t2, c, tol):
        q = convert_t_to_q(np.array([t0, t1, t2])) 
        return 1-1*float(c.evaluator().CheckSatisfied(q, tol))

col_func_handle_rational = partial(eval_cons_rational, c=collision_constraint, tol=0.01)

# Construct Rational Kinematics

In [5]:
from pydrake.all import RationalForwardKinematics
forward_kin = RationalForwardKinematics(plant)
t = forward_kin.t()
q_star = np.zeros(forward_kin.t().shape[0])
def convert_RationalForwardPoseList_to_TransformExpressionList(pose_list):
    ret = []
    for p in pose_list:
        ret.append(p.asRigidTransformExpr())
    return ret

def convert_t_to_q(t):
    q =np.arctan2(2*t/(1+t**2), (1-t**2)/(1+t**2))
    return q


def convert_q_to_t(q):
    t = np.sin(q)/(1+np.cos(q))
    return t

t_low = convert_q_to_t(q_low)
t_high = convert_q_to_t(q_high)


# IRIS IN T-SPACE CODE

In [6]:
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)


def TangentPlane(self, point):
    a = 2 * self.A().T @ self.A() @ (point - self.center())
    a = a / np.linalg.norm(a)
    b = a.dot(point)
    return a, b

Hyperellipsoid.TangentPlane = TangentPlane


query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))

sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()

# For SNOPT test.
snopt = SnoptSolver()

def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)


def GrowthVolumeRational(E, X_WA, X_WB, setA, setB, A, b, guess=None):
    snopt = SnoptSolver()
    prog = MathematicalProgram()
    t = forward_kin.t()
    prog.AddDecisionVariables(t)
    
    if guess is not None:
        prog.SetInitialGuess(t, guess)

    prog.AddLinearConstraint(A, b-np.inf, b, t)
    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)
    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    p_WA = X_WA.multiply(p_AA+0)

    p_WB = X_WB.multiply(p_BB+0)

    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)
    return result.is_success(), result.get_optimal_cost(), result.GetSolution(t)


In [7]:
def iris_rational_space(query, point, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    polytope_pullback_tol = .1
    ellipsoid_epsilon = 1e-1
    dim = plant.num_positions()
    lb = plant.GetPositionLowerLimits()
    rational_lb = forward_kin.ComputeTValue(lb, q_star)
    ub = plant.GetPositionUpperLimits()
    rational_ub = forward_kin.ComputeTValue(ub, q_star)
    volume_of_unit_sphere = 4.0*np.pi/3.0
    E = Hyperellipsoid(np.eye(3)/ellipsoid_epsilon, point)
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere
    
    link_poses_by_body_index_rat_pose = forward_kin.CalcLinkPoses(q_star, 
                                                         plant.world_body().index())
    X_WA_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)
    X_WB_list = convert_RationalForwardPoseList_to_TransformExpressionList(link_poses_by_body_index_rat_pose)

    inspector = query.inspector()
    pairs = inspector.GetCollisionCandidates()

    P = HPolyhedron.MakeBox(rational_lb, rational_ub)
    A = np.vstack((P.A(), np.zeros((10*len(pairs),3))))  # allow up to 10 faces per pair.
    b = np.concatenate((P.b(), np.zeros(10*len(pairs))))

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query, geom, inspector.GetFrameId(geom)) for geom in geom_ids}
    body_indexes_by_geom_id = {geom:
                               plant.GetBodyFromFrameId(inspector.GetFrameId(geom)).index() for geom in geom_ids} 
    
    #Turn onto true to certify regions using SOS at each iteration.
    certify = False
    # refine polytopes if not certified collision free
    refine = False and certify
        
    iteration = 0
    num_faces = 2*len(lb)
    while True:
        ## Find separating hyperplanes

        for geomA, geomB in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            # Run snopt at the beginning
            while True:
                X_WA = X_WA_list[int(body_indexes_by_geom_id[geomA])]
                X_WB = X_WB_list[int(body_indexes_by_geom_id[geomB])]
                success, growth, qstar = GrowthVolumeRational(E,
                    X_WA, X_WB,
                    sets[geomA], sets[geomB], A[:num_faces,:], b[:num_faces] - polytope_pullback_tol, point)
                if success:
                    print(f"snopt_example={qstar}, growth = {growth}")
                    # Add a face to the polytope
                    A[num_faces,:], b[num_faces] = E.TangentPlane(qstar)
                    num_faces += 1
                else:
                    break

            if certify:
                pass
            

        if any([np.any(A[:num_faces,:] @ t > b[:num_faces]) for t in require_containment_points]):
            print("terminating because a required containment point would have not been contained")
            break

        P = HPolyhedron(A[:num_faces,:],b[:num_faces])

        E = P.MaximumVolumeInscribedEllipsoid()
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = volume_of_unit_sphere / np.linalg.det(E.A())
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return P

In [8]:
#using hand-crafted seed points

seed_points = convert_q_to_t(np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]]) ) # passing


iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False



def iris_one_step(q_seed, iris_options = None):
    start_time = time.time()
#     hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    hpoly = iris_rational_space(query, q_seed, require_containment_points=[q_seed], iteration_limit=100)
  
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

def do_iris(seed_points, iris_options):
    regions = []
    ellipses = []
    for i in range(seed_points.shape[0]):
        ell, poly = iris_one_step(seed_points[i,:])
        ellipses.append(ell)
        regions.append(poly)



In [9]:
q_low_np = np.array(q_low)
q_high_np = np.array(q_high)

start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 6
seed_points = [start, target]

for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    regions.append(reg)
    ellipses.append(ell)
    
for _ in range(its):
    #rejection sampling to get initial feasible point 
    found = False
    while not found:
        t = np.random.rand(3)
        q_samp = (1-t)*q_low_np + t*q_high_np
        found = (col_func_handle(*q_samp)==0.0)
    print("point found: ", q_samp)
    ell, reg = do_iris(q_samp, iris_options)
    if ell.Volume() < 10.0:
        regions.append(reg)
        ellipses.append(ell)
    

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
# Solve path planning
start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])
start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity([.8, .8, .8])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj = spp.Solve()
print(time.time() - start_time)
print(traj.start_time())
print(traj.end_time())
    

In [23]:
print("done")

done
